In [169]:
#### Rama projektowa - Opisywanie obrazów z wykorzystaniem głębokiego uczenia
#### autorzy: Anna Konieczna, Mateusz Ostrowski

### Rama projektowa
Tutorial: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8 <br>
Zbiór danych: <br>
#### Opis:  <br>


In [170]:
#### Config
config = {"images_path": "../30k/flickr30k-images/",
             #"images_path": "../Flickr8k_Dataset/",
#            "train_images_path": "../Flickr8k_text/Flickr_8k.trainImages.txt", # file conatains the names of images to be used in train data
#            "test_images_path": "../Flickr8k_text/Flickr_8k.testImages.txt",
#            "train_path": "../Flickr8k_text/Flickr_8k.trainImages.txt", 
#            "token_path": "../Flickr8k_text/Flickr8k.token.txt",
           "train_images_path": "../30k/Flickr_30k.trainImages.txt", # file conatains the names of images to be used in train data
           "test_images_path": "../30k/Flickr_30k.testImages.txt",
           "train_path": "../30k/Flickr_30k.trainImages.txt", 
           "token_path": "../30k/Flickr30k.token.txt",
           "word_embedings_path": "../glove/glove.6B.200d.txt",
           "embedings_dim": 200, # Polish - 100
           "train_model": True, # True if you intend to train the model,
           "save_model": True,
           "encode_images": True,
           "pretrained_model_path": "",
           "encoded_images_test": "../Pickle/encoded_test_30_images.pkl",
           "encoded_images_train": "../Pickle/encoded_train_30_images.pkl",
           "report_path": "",
           "preprocessed_descriptions_save_path": "descriptions.txt",
           "use_lemma": False,
           "decay_lr": False,
           "spacy_lemma_model": "pl_spacy_model",
           "lstm_model_save_dir": "./models/",
           "lstm_model_save_path": "./models/model_dlr_ds_lstm.h5"}

In [171]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from keras import callbacks
from keras import optimizers

In [172]:
# Load the description set

def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

filename = config["token_path"]
# załaduj opisy
doc = load_doc(filename)
print(doc[:300])

1000092795.jpg#0	Two young guys with shaggy hair look at their hands while hanging out in the yard .
1000092795.jpg#1	Two young , White males are outside near many bushes .
1000092795.jpg#2	Two men in green shirts are standing in a yard .
1000092795.jpg#3	A man in a blue shirt standing in a garden .


In [173]:
# Load the descriptions to dict structure

def load_descriptions(doc):
	mapping = dict()
	for line in doc.split('\n'):
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 31783 


In [174]:
if config["use_lemma"]:
    import spacy
    nlp = spacy.load('pl_spacy_model')

def clean_descriptions(descriptions):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc = desc_list[i]
      if config["use_lemma"]:
          doc = nlp(desc)
      # tokenize
      desc = desc.split()
      # convert to lower case
      desc = [word.lower() for word in desc]
# Lematyzacja
#### spacy model from here: http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=view&target=pl_spacy_model-0.1.0.tar.gz
#### spacy dokumentacja - https://spacy.io/usage
      if config["use_lemma"]:
          desc = [word.lemma_ for word in doc]
      # remove punctuation from each token
      desc = [w.translate(table) for w in desc]
      # remove tokens with numbers in them
      desc = [word for word in desc if word.isalpha()]
      # store as string
      desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [175]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 19758


In [176]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, config["preprocessed_descriptions_save_path"])

In [177]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = config["train_path"]
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 21294


In [178]:
# Below path contains all the images
images = config["images_path"]
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [179]:
img

['../30k/flickr30k-images/4802006144.jpg',
 '../30k/flickr30k-images/3603879637.jpg',
 '../30k/flickr30k-images/2825668136.jpg',
 '../30k/flickr30k-images/4547845821.jpg',
 '../30k/flickr30k-images/3084149186.jpg',
 '../30k/flickr30k-images/4590206492.jpg',
 '../30k/flickr30k-images/3339558806.jpg',
 '../30k/flickr30k-images/3639684919.jpg',
 '../30k/flickr30k-images/3497069793.jpg',
 '../30k/flickr30k-images/5352158457.jpg',
 '../30k/flickr30k-images/3517370470.jpg',
 '../30k/flickr30k-images/5708913719.jpg',
 '../30k/flickr30k-images/3375070563.jpg',
 '../30k/flickr30k-images/7081584189.jpg',
 '../30k/flickr30k-images/2538588862.jpg',
 '../30k/flickr30k-images/4637704411.jpg',
 '../30k/flickr30k-images/953120250.jpg',
 '../30k/flickr30k-images/503486811.jpg',
 '../30k/flickr30k-images/4584267739.jpg',
 '../30k/flickr30k-images/264483970.jpg',
 '../30k/flickr30k-images/2339140905.jpg',
 '../30k/flickr30k-images/149150776.jpg',
 '../30k/flickr30k-images/314567175.jpg',
 '../30k/flickr3

In [180]:
# Read the train image names in a set
train_images = set(open(config["train_images_path"], 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [181]:
# Below file conatains the names of images to be used in test data
test_images_file = config["test_images_path"]
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [182]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions(config["preprocessed_descriptions_save_path"], train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=21294


In [183]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [184]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [185]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [186]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [187]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
if config["encode_images"]:
    start = time()
    encoding_train = {}
    for img in train_img:
        print(img)
        encoding_train[img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

../30k/flickr30k-images/3603879637.jpg
.../30k/flickr30k-images/2825668136.jpg
.../30k/flickr30k-images/4547845821.jpg
.../30k/flickr30k-images/3084149186.jpg
.../30k/flickr30k-images/4590206492.jpg
.../30k/flickr30k-images/3339558806.jpg
.../30k/flickr30k-images/3639684919.jpg
.../30k/flickr30k-images/3497069793.jpg
.../30k/flickr30k-images/5352158457.jpg
.../30k/flickr30k-images/3375070563.jpg
.../30k/flickr30k-images/2538588862.jpg
.../30k/flickr30k-images/4637704411.jpg
.../30k/flickr30k-images/503486811.jpg
.../30k/flickr30k-images/264483970.jpg
.../30k/flickr30k-images/149150776.jpg
.../30k/flickr30k-images/314567175.jpg
.../30k/flickr30k-images/533124119.jpg
.../30k/flickr30k-images/2101068655.jpg
.../30k/flickr30k-images/457945610.jpg
.../30k/flickr30k-images/4549569635.jpg
.../30k/flickr30k-images/5869269064.jpg
.../30k/flickr30k-images/4982004472.jpg
.../30k/flickr30k-images/508958120.jpg
.../30k/flickr30k-images/210593985.jpg
.../30k/flickr30k-images/75902338.jpg
.../30k/fli

.../30k/flickr30k-images/5925545440.jpg
.../30k/flickr30k-images/4930823945.jpg
.../30k/flickr30k-images/4567985140.jpg
.../30k/flickr30k-images/1022454428.jpg
.../30k/flickr30k-images/3669527584.jpg
.../30k/flickr30k-images/408695996.jpg
.../30k/flickr30k-images/1833327575.jpg
.../30k/flickr30k-images/4468251915.jpg
.../30k/flickr30k-images/3916066890.jpg
.../30k/flickr30k-images/3222496967.jpg
.../30k/flickr30k-images/2148234367.jpg
.../30k/flickr30k-images/4781282778.jpg
.../30k/flickr30k-images/3798453280.jpg
.../30k/flickr30k-images/3010404509.jpg
.../30k/flickr30k-images/2957681960.jpg
.../30k/flickr30k-images/3970078088.jpg
.../30k/flickr30k-images/5747873823.jpg
.../30k/flickr30k-images/3295391572.jpg
.../30k/flickr30k-images/184855171.jpg
.../30k/flickr30k-images/336551615.jpg
.../30k/flickr30k-images/2113124815.jpg
.../30k/flickr30k-images/3568197730.jpg
.../30k/flickr30k-images/2853407781.jpg
.../30k/flickr30k-images/4621219089.jpg
.../30k/flickr30k-images/3735197045.jpg
...

.../30k/flickr30k-images/5742849511.jpg
.../30k/flickr30k-images/4972936619.jpg
.../30k/flickr30k-images/4946160196.jpg
.../30k/flickr30k-images/5870019388.jpg
.../30k/flickr30k-images/3624076529.jpg
.../30k/flickr30k-images/4612831165.jpg
.../30k/flickr30k-images/7162734265.jpg
.../30k/flickr30k-images/4813123584.jpg
.../30k/flickr30k-images/3753939.jpg
.../30k/flickr30k-images/2807437977.jpg
.../30k/flickr30k-images/27590618.jpg
.../30k/flickr30k-images/3411712848.jpg
.../30k/flickr30k-images/3143754462.jpg
.../30k/flickr30k-images/3693297007.jpg
.../30k/flickr30k-images/3240519143.jpg
.../30k/flickr30k-images/2103080234.jpg
.../30k/flickr30k-images/4908009544.jpg
.../30k/flickr30k-images/382151094.jpg
.../30k/flickr30k-images/2494088238.jpg
.../30k/flickr30k-images/3524671328.jpg
.../30k/flickr30k-images/4467543993.jpg
.../30k/flickr30k-images/229954612.jpg
.../30k/flickr30k-images/2716718260.jpg
.../30k/flickr30k-images/24815360.jpg
.../30k/flickr30k-images/4590153852.jpg
.../30k/f

.../30k/flickr30k-images/7237669608.jpg
.../30k/flickr30k-images/360275648.jpg
.../30k/flickr30k-images/4771247095.jpg
.../30k/flickr30k-images/3371279606.jpg
.../30k/flickr30k-images/23010569.jpg
.../30k/flickr30k-images/5379229830.jpg
.../30k/flickr30k-images/2480820830.jpg
.../30k/flickr30k-images/1801188148.jpg
.../30k/flickr30k-images/4470667650.jpg
.../30k/flickr30k-images/3043766.jpg
.../30k/flickr30k-images/4876652032.jpg
.../30k/flickr30k-images/4857809566.jpg
.../30k/flickr30k-images/3051125715.jpg
.../30k/flickr30k-images/4303243256.jpg
.../30k/flickr30k-images/1881494074.jpg
.../30k/flickr30k-images/2231193384.jpg
.../30k/flickr30k-images/4902416291.jpg
.../30k/flickr30k-images/863036078.jpg
.../30k/flickr30k-images/6780320278.jpg
.../30k/flickr30k-images/1432179046.jpg
.../30k/flickr30k-images/2809793070.jpg
.../30k/flickr30k-images/5949976408.jpg
.../30k/flickr30k-images/7165826558.jpg
.../30k/flickr30k-images/4882073197.jpg
.../30k/flickr30k-images/5218695756.jpg
.../30k

.../30k/flickr30k-images/3981584370.jpg
.../30k/flickr30k-images/3389037189.jpg
.../30k/flickr30k-images/3459871361.jpg
.../30k/flickr30k-images/4764082600.jpg
.../30k/flickr30k-images/493274225.jpg
.../30k/flickr30k-images/450596617.jpg
.../30k/flickr30k-images/2326133103.jpg
.../30k/flickr30k-images/3641999223.jpg
.../30k/flickr30k-images/680790383.jpg
.../30k/flickr30k-images/3951142610.jpg
.../30k/flickr30k-images/123728333.jpg
.../30k/flickr30k-images/6194677011.jpg
.../30k/flickr30k-images/4909077063.jpg
.../30k/flickr30k-images/6029710737.jpg
.../30k/flickr30k-images/4862041366.jpg
.../30k/flickr30k-images/4736257831.jpg
.../30k/flickr30k-images/2385034500.jpg
.../30k/flickr30k-images/3335997221.jpg
.../30k/flickr30k-images/3980305243.jpg
.../30k/flickr30k-images/4825747859.jpg
.../30k/flickr30k-images/199463720.jpg
.../30k/flickr30k-images/1553023640.jpg
.../30k/flickr30k-images/5400154.jpg
.../30k/flickr30k-images/2924312981.jpg
.../30k/flickr30k-images/2833502675.jpg
.../30k/

...

.../30k/flickr30k-images/1453251773.jpg
.../30k/flickr30k-images/4883101538.jpg
.../30k/flickr30k-images/3214573346.jpg
.../30k/flickr30k-images/4922043349.jpg
.../30k/flickr30k-images/3353036763.jpg
.../30k/flickr30k-images/432167214.jpg
.../30k/flickr30k-images/2362918118.jpg
.../30k/flickr30k-images/470887795.jpg
.../30k/flickr30k-images/6553168529.jpg
.../30k/flickr30k-images/2857609295.jpg
.../30k/flickr30k-images/4978966549.jpg
.../30k/flickr30k-images/4851249911.jpg
.../30k/flickr30k-images/6761660155.jpg
.../30k/flickr30k-images/2545282421.jpg
.../30k/flickr30k-images/137430875.jpg
.../30k/flickr30k-images/275170947.jpg
.../30k/flickr30k-images/2071705189.jpg
.../30k/flickr30k-images/3004823335.jpg
.../30k/flickr30k-images/2230363312.jpg
.../30k/flickr30k-images/6079467981.jpg
.../30k/flickr30k-images/3500505549.jpg
.../30k/flickr30k-images/6979707950.jpg
.../30k/flickr30k-images/8217641081.jpg
.../30k/flickr30k-images/4116166193.jpg
.../30k/flickr30k-images/7194592318.jpg
.../

.../30k/flickr30k-images/4587837067.jpg
.../30k/flickr30k-images/2353830402.jpg
.../30k/flickr30k-images/1136758631.jpg
.../30k/flickr30k-images/2667092132.jpg
.../30k/flickr30k-images/4689003496.jpg
.../30k/flickr30k-images/5944781298.jpg
.../30k/flickr30k-images/2536850983.jpg
.../30k/flickr30k-images/2797438951.jpg
.../30k/flickr30k-images/1362206741.jpg
.../30k/flickr30k-images/6276644870.jpg
.../30k/flickr30k-images/6034056358.jpg
.../30k/flickr30k-images/1472053993.jpg
.../30k/flickr30k-images/201214645.jpg
.../30k/flickr30k-images/3442978981.jpg
.../30k/flickr30k-images/4812991186.jpg
.../30k/flickr30k-images/3194134352.jpg
.../30k/flickr30k-images/4612281699.jpg
.../30k/flickr30k-images/164380865.jpg
.../30k/flickr30k-images/301655083.jpg
.../30k/flickr30k-images/3268727984.jpg
.../30k/flickr30k-images/214774826.jpg
.../30k/flickr30k-images/7922678762.jpg
.../30k/flickr30k-images/7432698546.jpg
.../30k/flickr30k-images/2475640704.jpg
.../30k/flickr30k-images/338857661.jpg
.../3

.../30k/flickr30k-images/2215767852.jpg
.../30k/flickr30k-images/7817889978.jpg
.../30k/flickr30k-images/4688301633.jpg
.../30k/flickr30k-images/2172493537.jpg
.../30k/flickr30k-images/160566014.jpg
.../30k/flickr30k-images/4754011554.jpg
.../30k/flickr30k-images/1476002408.jpg
.../30k/flickr30k-images/3328296271.jpg
.../30k/flickr30k-images/2301525531.jpg
.../30k/flickr30k-images/1775741422.jpg
.../30k/flickr30k-images/3629615546.jpg
.../30k/flickr30k-images/2463067409.jpg
.../30k/flickr30k-images/4433634883.jpg
.../30k/flickr30k-images/1056338697.jpg
.../30k/flickr30k-images/3952964772.jpg
.../30k/flickr30k-images/4825458628.jpg
.../30k/flickr30k-images/5352303362.jpg
.../30k/flickr30k-images/463476205.jpg
.../30k/flickr30k-images/4702851298.jpg
.../30k/flickr30k-images/3015891201.jpg
.../30k/flickr30k-images/2166529027.jpg
.../30k/flickr30k-images/395461421.jpg
.../30k/flickr30k-images/5359437945.jpg
.../30k/flickr30k-images/4656684238.jpg
.../30k/flickr30k-images/63108800.jpg
.../3

.../30k/flickr30k-images/4288768841.jpg
.../30k/flickr30k-images/2617219856.jpg
.../30k/flickr30k-images/275171617.jpg
.../30k/flickr30k-images/4668692006.jpg
.../30k/flickr30k-images/3133825703.jpg
.../30k/flickr30k-images/7363849060.jpg
.../30k/flickr30k-images/2840344516.jpg
.../30k/flickr30k-images/4945381466.jpg
.../30k/flickr30k-images/944788251.jpg
.../30k/flickr30k-images/3524436870.jpg
.../30k/flickr30k-images/3069282021.jpg
.../30k/flickr30k-images/4730009983.jpg
.../30k/flickr30k-images/3082997532.jpg
.../30k/flickr30k-images/4668777411.jpg
.../30k/flickr30k-images/3242919570.jpg
.../30k/flickr30k-images/4533155215.jpg
.../30k/flickr30k-images/2687672606.jpg
.../30k/flickr30k-images/2866016431.jpg
.../30k/flickr30k-images/1474240647.jpg
.../30k/flickr30k-images/4923233692.jpg
.../30k/flickr30k-images/6778210987.jpg
.../30k/flickr30k-images/2721656220.jpg
.../30k/flickr30k-images/4689717802.jpg
.../30k/flickr30k-images/434642339.jpg
.../30k/flickr30k-images/2451285022.jpg
...

.../30k/flickr30k-images/3091916691.jpg
.../30k/flickr30k-images/316293396.jpg
.../30k/flickr30k-images/4567711312.jpg
.../30k/flickr30k-images/2866093652.jpg
.../30k/flickr30k-images/2911928620.jpg
.../30k/flickr30k-images/217388728.jpg
.../30k/flickr30k-images/2887750774.jpg
.../30k/flickr30k-images/3110840233.jpg
.../30k/flickr30k-images/3386375153.jpg
.../30k/flickr30k-images/651277216.jpg
.../30k/flickr30k-images/124195430.jpg
.../30k/flickr30k-images/3790770806.jpg
.../30k/flickr30k-images/241345905.jpg
.../30k/flickr30k-images/1470132731.jpg
.../30k/flickr30k-images/1801874841.jpg
.../30k/flickr30k-images/2525232298.jpg
.../30k/flickr30k-images/2045444296.jpg
.../30k/flickr30k-images/4851246557.jpg
.../30k/flickr30k-images/2987121689.jpg
.../30k/flickr30k-images/864827397.jpg
.../30k/flickr30k-images/2099140476.jpg
.../30k/flickr30k-images/161669933.jpg
.../30k/flickr30k-images/2667080272.jpg
.../30k/flickr30k-images/2716903793.jpg
.../30k/flickr30k-images/2969233171.jpg
.../30k

In [188]:
encoding_train

{'3603879637.jpg': array([0.72607243, 0.62489974, 1.5182153 , ..., 0.66608214, 0.9235857 ,
        0.        ], dtype=float32),
 '2825668136.jpg': array([0.12888508, 0.45360592, 0.10436323, ..., 0.29842684, 0.5230663 ,
        0.10125212], dtype=float32),
 '4547845821.jpg': array([0.07572428, 0.3099772 , 0.20990682, ..., 0.06078173, 0.4273753 ,
        0.6126987 ], dtype=float32),
 '3084149186.jpg': array([0.26059324, 0.07451665, 0.10219958, ..., 0.58292264, 2.4048266 ,
        0.5231879 ], dtype=float32),
 '4590206492.jpg': array([0.9811406 , 0.23767132, 0.49787557, ..., 1.0547119 , 0.41132975,
        0.4514228 ], dtype=float32),
 '3339558806.jpg': array([0.59399253, 0.7709943 , 0.85092276, ..., 0.933331  , 0.3824733 ,
        0.01447037], dtype=float32),
 '3639684919.jpg': array([0.87815076, 0.63673013, 1.7742828 , ..., 0.23591831, 0.67857707,
        0.37089407], dtype=float32),
 '3497069793.jpg': array([0.1956791 , 0.25681615, 0.25072768, ..., 1.031447  , 0.0682385 ,
        1.023

In [189]:
import os
import pickle

if config["encode_images"]:
    mode = 'a' if os.path.exists(config["encoded_images_train"]) else 'wb'

    # Save the bottleneck train features to disk
    with open(config["encoded_images_train"], "wb") as encoded_pickle:
        pickle.dump(encoding_train, encoded_pickle)

In [190]:
# Call the funtion to encode all the test images - Execute this only once
if config["encode_images"]:
    start = time()
    encoding_test = {}
    for img in test_img:
        encoding_test[img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [191]:
# Save the bottleneck test features to disk
if config["encode_images"]:
    with open(config["encoded_images_test"], "wb") as encoded_pickle:
        pickle.dump(encoding_test, encoded_pickle)

In [192]:
with open(config["encoded_images_train"], "rb") as encoded_pickle:
    encoding_train = load(encoded_pickle)
with open(config["encoded_images_test"], "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [193]:
train_features = load(open(config["encoded_images_train"], "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=21294


In [194]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

106470

In [195]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 16835 -> 4448


In [196]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [197]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

4449

In [198]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 80


In [199]:
data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)

<generator object data_generator at 0x7fab8bdae820>

In [200]:
train_features

{'3603879637.jpg': array([0.72607243, 0.62489974, 1.5182153 , ..., 0.66608214, 0.9235857 ,
        0.        ], dtype=float32),
 '2825668136.jpg': array([0.12888508, 0.45360592, 0.10436323, ..., 0.29842684, 0.5230663 ,
        0.10125212], dtype=float32),
 '4547845821.jpg': array([0.07572428, 0.3099772 , 0.20990682, ..., 0.06078173, 0.4273753 ,
        0.6126987 ], dtype=float32),
 '3084149186.jpg': array([0.26059324, 0.07451665, 0.10219958, ..., 0.58292264, 2.4048266 ,
        0.5231879 ], dtype=float32),
 '4590206492.jpg': array([0.9811406 , 0.23767132, 0.49787557, ..., 1.0547119 , 0.41132975,
        0.4514228 ], dtype=float32),
 '3339558806.jpg': array([0.59399253, 0.7709943 , 0.85092276, ..., 0.933331  , 0.3824733 ,
        0.01447037], dtype=float32),
 '3639684919.jpg': array([0.87815076, 0.63673013, 1.7742828 , ..., 0.23591831, 0.67857707,
        0.37089407], dtype=float32),
 '3497069793.jpg': array([0.1956791 , 0.25681615, 0.25072768, ..., 1.031447  , 0.0682385 ,
        1.023

In [201]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
                yield ([array(X1), array(X2)], array(y)) 
                X1, X2, y = list(), list(), list()
                n=0

In [202]:
# Load Glove vectors

embeddings_index = {} # empty dictionary

f = open(config["word_embedings_path"], encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [203]:
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, config["embedings_dim"]))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [204]:
embedding_matrix.shape

(4449, 200)

In [205]:
# Deep stacked LSTM
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, config["embedings_dim"], mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256, return_sequences=True, input_shape=(2048,))(se2)
se4 = LSTM(256, return_sequences=True)(se3)
se5 = LSTM(256)(se4)
decoder1 = add([fe2, se5])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [206]:
# inputs1 = Input(shape=(2048,))
# fe1 = Dropout(0.5)(inputs1)
# fe2 = Dense(256, activation='relu')(fe1)
# inputs2 = Input(shape=(max_length,))
# se1 = Embedding(vocab_size, config["embedings_dim"], mask_zero=True)(inputs2)
# se2 = Dropout(0.5)(se1)
# se3 = LSTM(256)(se2)

# decoder1 = add([fe2, se3])
# decoder2 = Dense(256, activation='relu')(decoder1)
# outputs = Dense(vocab_size, activation='softmax')(decoder2)
# model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [207]:
model.summary()

Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 80, 200)      889800      input_14[0][0]                   
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 80, 200)      0           embedding_3[0][0]                
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 2048)]       0                                            
______________________________________________________________________________________

In [208]:
model.layers[1]

In [209]:
model.layers[1].set_weights([embedding_matrix])
model.layers[1].trainable = False

In [210]:
if config["decay_lr"]:
    opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
else:    
    opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [211]:
if config["train_model"]:
    epochs = 100
    number_pics_per_bath = 3
    steps = len(train_descriptions)//number_pics_per_bath

In [212]:
if config["train_model"]:
    callback = callbacks.EarlyStopping(monitor='loss',min_delta = 0.001, patience=1)
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=epochs, steps_per_epoch=steps, callbacks=[callback], verbose=1)
    if config["save_model"]:
        writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
        mode = 'a' if os.path.exists(writepath) else 'wb'
        model.save(writepath)

Epoch 1/100
7098/7098 [==============================] - 5943s 837ms/step - loss: 3.8242
Epoch 2/100
7098/7098 [==============================] - 5943s 837ms/step - loss: 3.2482
Epoch 3/100
7098/7098 [==============================] - 5938s 837ms/step - loss: 3.0945
Epoch 4/100
7098/7098 [==============================] - 5933s 836ms/step - loss: 3.0108
Epoch 5/100
7098/7098 [==============================] - 5931s 836ms/step - loss: 2.9556
Epoch 6/100
7098/7098 [==============================] - 5928s 835ms/step - loss: 2.9164
Epoch 7/100
7098/7098 [==============================] - 5928s 835ms/step - loss: 2.8849
Epoch 8/100
7098/7098 [==============================] - 5928s 835ms/step - loss: 2.8608
Epoch 9/100
7098/7098 [==============================] - 5931s 836ms/step - loss: 2.8392
Epoch 10/100
7098/7098 [==============================] - 5934s 836ms/step - loss: 2.8241
Epoch 11/100
7098/7098 [==============================] - 5931s 836ms/step - loss: 2.8083
Epoch 12/100
7098/7

In [215]:
writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
mode = 'a' if os.path.exists(writepath) else 'wb'
model.save(writepath)

In [218]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(config["lstm_model_save_dir"], monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

In [ ]:
if config["train_model"]:
    callback = callbacks.EarlyStopping(monitor='loss',min_delta = 0.001, patience=2)
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=epochs, steps_per_epoch=steps, callbacks=[callback, checkpoint], verbose=1)
    if config["save_model"]:
        writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
        mode = 'a' if os.path.exists(writepath) else 'wb'
        model.save(writepath)

Epoch 1/100
 202/3549 [>.............................] - ETA: 1:19:25 - loss: 2.3651

In [ ]:
# if config["train_model"]:
#     #model.optimizer.lr = 0.0001
#     epochs = 100
#     number_pics_per_bath = 6
#     steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
# if config["train_model"]:
# #     for i in range(epochs):
#     generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
#     model.fit_generator(generator, epochs=epochs, steps_per_epoch=steps, callbacks=[callback], verbose=1)
#     #model.save('./model_weights/model_' + str(i) + '.h5')

In [ ]:
if config["save_model"]:
    writepath = config["lstm_model_save_dir"] + 'model_' + str(i) + '.h5'
    mode = 'a' if os.path.exists(writepath) else 'wb'
    model.save(writepath)

In [ ]:
images = config["images_path"]

In [ ]:
with open(config["encoded_images_test"], "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
for i in range(5, 10):
    pic = list(encoding_test.keys())[i]
    image = encoding_test[pic].reshape((1,2048))
    x=plt.imread(images+pic)
    plt.imshow(x)
    plt.show()
    print("Greedy:",greedySearch(image))
    print(descriptions[pic[:-4]])

In [ ]:
#### Metrics
##### Bag of words = bow

def get_bow(description):
    bow = description.split(' ')
    bow = set(bow)
    return bow

#calculate bow score
def bow_score(bow_train, bow_pred):
    overlap = bow_train & bow_pred
    universe = bow_train | bow_pred

    result1 = float(len(overlap)) / len(bow_train) * 100
    result2 = float(len(overlap)) / len(bow_pred) * 100
    result3 = float(len(overlap)) / len(universe) * 100
    return result1, len(overlap)

In [ ]:

##### Cider
#function for calculating cider metric
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cider(given_captions, prediction):
    prediction_arr = []
    prediction_arr.append(prediction)
    vectorizer = TfidfVectorizer()
    trainedV = vectorizer.fit(given_captions)
    givenTfIdf = trainedV.transform(given_captions).toarray()
    predTfIdf = trainedV.transform(prediction_arr).toarray()
    partialCiders = 0
    help_arr = []
    
    for c in givenTfIdf:
        help_arr.append(c)
        partialCiders = partialCiders + cosine_similarity(predTfIdf,help_arr)
        help_arr = []
        
    return partialCiders/len(givenTfIdf)

In [ ]:
#### 
# import warnings; warnings.simplefilter('ignore')
# from IPython.display import clear_output

# #preparation of metrics lists
# bleu_train_scores = []
# meteor_train_scores = []
# cider_train_scores = []
    
# train_pics = list(encoding_train.keys())

# #calculation of metrics for train images dataset
# for j in range(0,len(train_pics)):
#     pic = train_pics[j]
#     image = encoding_train[pic].reshape((1,2048))

#     actual_desc_0 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][0]
#     actual_desc_1 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][1]
#     actual_desc_2 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][2]
#     actual_desc_3 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][3]
#     actual_desc_4 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][4]

#     reference = [actual_desc_0.split(),
#          actual_desc_1.split(),
#          actual_desc_2.split(),
#          actual_desc_3.split(),
#          actual_desc_4.split()]

#     generated = greedySearch(image)
#     bleu_train_scores.append(sentence_bleu(reference, generated.split(), weights=(1, 0, 0, 0)))
#     meteor_train_scores.append(meteor_score(descriptions[pic[:-4]],generated))
#     cider_train_scores.append(cider(descriptions[pic[:-4]],generated)[0][0])

#     clear_output(wait=True)
#     print('train',j,len(train_pics), flush=True)



In [ ]:
from numpy import mean

result_data = {}

#calculate mean score based on socres list for each metric

result_data = [
    mean(bleu_train_scores),
    mean(meteor_train_scores),
    mean(cider_train_scores),
]

print(['Bleu TREN', 'Meteor TREN','CIDEr TREN'])
result_data

In [ ]:
from tqdm.notebook import tqdm

#preparation of metrics lists
bleu_test_scores = []
meteor_test_scores = []
cider_test_scores = []
test_pics = list(encoding_test.keys())

#calculation of metrics for test images dataset
#for j in tqdm(range(0,len(test_pics))):
for j in tqdm(range(0,1000)):
    pic = test_pics[j]
    image = encoding_test[pic].reshape((1,2048))

    actual_desc_0 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][0]
    actual_desc_1 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][1]
    actual_desc_2 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][2]
    actual_desc_3 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][3]
    actual_desc_4 = descriptions[pic.rsplit( ".", 1 )[ 0 ]][4]

    reference = [actual_desc_0.split(),
                 actual_desc_1.split(),
                 actual_desc_2.split(),
                 actual_desc_3.split(),
                 actual_desc_4.split()]

    generated = greedySearch(image)

    bleu_test_scores.append(sentence_bleu(reference, generated.split(), weights=(1, 0, 0, 0)))
    meteor_test_scores.append(meteor_score(descriptions[pic[:-4]],generated))
    cider_test_scores.append(cider(descriptions[pic[:-4]],generated)[0][0])

    clear_output(wait=True)
    print('test', j, len(test_pics), flush=True)



In [ ]:
#calculate mean score based on socres list for each metric
from numpy import mean

result_data = {}

# for i in range(0,5):
result_data = [
    mean(bleu_test_scores),
    mean(meteor_test_scores),
    mean(cider_test_scores)
]

print(['Bleu TEST', 'Meteor TEST', 'CIDEr TEST'])
result_data

Early stopping patience=3, delta = 0.001, bleu weights=(0.25, 0.25, 0.25, 0.25)

['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.08826357992794069, 0.3812428392811727, 0.2538227205501579]

['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.06709583754955759, 0.35676080070908517, 0.24570790686788618]


Early stopping patience=3, delta = 0.001,
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.5314928729248558, 0.35676080070908517, 0.24570790686788618]

Decaying learning rate
['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.5389221988318406, 0.36500372248553375, 0.246789750564132]<>
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.5099813205752186, 0.3409624319787307, 0.23597343552878425]

Przy overfittingu wygenerowane zdania są dłuższe/bardziej złożone, lecz mniej wiernie opisują obrazek. Np.:
reedy: a man in a black shirt is sitting on a bench in front of a brick building
['a japanese swordsman jumps into the air', 'a man in a blue outfit is swinging a sword in the air', 'an actor dressed as a pirate performs in an outdoor park', 'a person wearing a blue robe is in the air holding a sword', 'man in martial artsstyle costume jumping in midair']

Na obrazku nie było ani ławki ani budynku.

### Deep Stacked LSTM + decaying learning rate
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.4791308028567803, 0.31819220287157657, 0.22750624379012033]

### Early stopping DS-LSTM
['Bleu TEST', 'Meteor TEST', 'CIDEr TEST']
[0.4811540307541167, 0.32533699474459354, 0.22786212254431967]
['Bleu TREN', 'Meteor TREN', 'CIDEr TREN']
[0.5059629215074034, 0.3502870455324523, 0.23924489269523558]